In [22]:
window_size = 20
input_dim = 152
output_dim = 34 
hidden_dim = 128
batch_size = 32
num_epochs = 10
num_layers = 2

In [2]:
import torch
import torch.nn as nn
import numpy as np
import os
import pandas as pd
import dask.dataframe as dd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from IPython.display import clear_output
import time

In [10]:
class MyDataset(Dataset):
    def __init__(self, file_path, chunk_size):
        self.file_path = file_path
        self.chunk_size = chunk_size
        
        self.columns = pd.read_csv(self.file_path, nrows=0).columns.str.strip().str.replace('\xa0', ' ').tolist()
        self.train_columns = [col for col in self.columns if col != 'Discard']  # 排除 'Discard' 欄位
        self.num_samples = sum(1 for _ in open(self.file_path, encoding='utf-8')) - 1  # 計算總樣本數
        
    def __getitem__(self, idx):
        chunk_start = idx // self.chunk_size * self.chunk_size
        df = pd.read_csv(self.file_path, skiprows=chunk_start + 1, nrows=self.chunk_size, header=None, encoding='utf-8')  # 跳過標題和之前的行

        df.columns = self.columns
        
        if 'Discard' not in df.columns:
            raise KeyError(f"Chunk starting at row {chunk_start + 1} does not contain 'Discard' column.")
        
            
        sample_idx = idx % self.chunk_size
        
        train_data = df[self.train_columns].iloc[sample_idx].values
        
        value_data = df['Discard'].iloc[sample_idx]  # 提取 'Discard' 欄位
        
        return torch.tensor(train_data, dtype=torch.float32), torch.tensor(value_data, dtype=torch.float32)
        
    def __len__(self):
        return self.num_samples

In [4]:
"""
讀取csv
"""
file_path = 'E:/專題/data/2021/DiscardData.csv'
dataset = MyDataset(file_path, chunk_size=50000)  # 每次只加載 50000 行
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [25]:
class MyMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [27]:
model = MyMLP(input_dim, hidden_dim, output_dim).to(device)
loss_criterion  = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)

In [32]:
current_loss = 0
for epoch in range(num_epochs):
    for train_data, value_data in train_dataloader:
        model.train()
        train_data, value_data = train_data.to(device), value_data.to(device)

        outputs = model(train_data)
        value_data = value_data.long()
        probabilities = F.softmax(outputs, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)
        correct_predictions = (predicted_labels == value_data).sum().item()
        accuracy = (correct_predictions / value_data.size(0))*100
        
        loss = loss_criterion(outputs, value_data)
        current_loss = loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        clear_output()
        print(predicted_labels)
        print(value_data)
        print(f"Correct Predictions: {correct_predictions}")
        print(f"Total Predictions: {value_data.size(0)}")
        print(f"Accuracy: {accuracy:.4f}","%")
        print(f"At epoch: {epoch}, loss: {current_loss}")
        for name, param in model.named_parameters():
            if param.grad is not None:
                print(f'{name} gradient norm: {param.grad.norm()}')

tensor([29, 30, 29, 29, 30, 29, 29, 29, 30, 29, 29, 29, 30, 29, 30, 29, 26, 29,
        18, 29, 26, 29, 18, 31, 26, 31, 26,  8, 26,  8, 26,  8],
       device='cuda:0')
tensor([ 8, 27, 33, 31, 32, 29, 29, 16, 28,  0, 31, 13, 18, 21, 27, 21, 19, 20,
        18, 31,  6, 23, 18, 22,  0, 28, 11, 26, 27, 32,  8, 20],
       device='cuda:0')
Correct Predictions: 4
Total Predictions: 32
Accuracy: 12.5000 %
At epoch: 0, loss: 3.291640043258667
fc1.weight gradient norm: 3.827540874481201
fc1.bias gradient norm: 0.006986912339925766
fc2.weight gradient norm: 0.8734406232833862
fc2.bias gradient norm: 0.01361778937280178
fc3.weight gradient norm: 1.3633328676223755
fc3.bias gradient norm: 0.16168561577796936


KeyboardInterrupt: 

In [ ]:
current_loss = 0
for epoch in range(num_epochs):
    for train_data, value_data in train_dataloader:
        model.train()
        train_data, value_data = train_data.to(device), value_data.to(device)

        outputs = model(train_data)
        value_data = value_data.long()
        probabilities = F.softmax(outputs, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)
        correct_predictions = (predicted_labels == value_data).sum().item()
        accuracy = (correct_predictions / value_data.size(0))*100
        
        loss = loss_criterion(outputs, value_data)
        current_loss = loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        clear_output()
        print(predicted_labels)
        print(value_data)
        print(f"Correct Predictions: {correct_predictions}")
        print(f"Total Predictions: {value_data.size(0)}")
        print(f"Accuracy: {accuracy:.4f}","%")
        print(f"At epoch: {epoch}, loss: {current_loss}")
        for name, param in model.named_parameters():
            if param.grad is not None:
                print(f'{name} gradient norm: {param.grad.norm()}')

tensor([29, 30, 29, 29, 30, 29, 29, 29, 30, 29, 29, 29, 30, 29, 30, 29, 26, 29,
        18, 29, 26, 29, 18, 31, 26, 31, 26,  8, 26,  8, 26,  8],
       device='cuda:0')
tensor([ 8, 27, 33, 31, 32, 29, 29, 16, 28,  0, 31, 13, 18, 21, 27, 21, 19, 20,
        18, 31,  6, 23, 18, 22,  0, 28, 11, 26, 27, 32,  8, 20],
       device='cuda:0')
Correct Predictions: 4
Total Predictions: 32
Accuracy: 12.5000 %
At epoch: 0, loss: 3.291640043258667
fc1.weight gradient norm: 3.827540874481201
fc1.bias gradient norm: 0.006986912339925766
fc2.weight gradient norm: 0.8734406232833862
fc2.bias gradient norm: 0.01361778937280178
fc3.weight gradient norm: 1.3633328676223755
fc3.bias gradient norm: 0.16168561577796936


KeyboardInterrupt: 